<a href="https://colab.research.google.com/github/prometheusDE/mylastai/blob/master/mylastai_0008_project_handlevideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


1. Install Tensorflow and Tensorflow Dataset Extension

In [0]:
%%capture
!pip install tensorflow-datasets
!pip install tensorflow

!nvcc --version

!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist/cuda/10.0 nvidia-dali
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist/cuda/10.0 nvidia-dali-tf-plugin

2. Handle Video File


In [0]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
import numpy as np
from timeit import default_timer as timer
import numpy as np
import matplotlib.pyplot as plt
import os.path

# https://docs.nvidia.com/deeplearning/sdk/dali-developer-guide/docs/examples/sequence_processing/video/video_reader_simple_example.html

In [0]:
batch_size=2
sequence_length=8
initial_prefetch_size=16
video_directory = '/content/gdrive/My Drive/CycleAI/20200308'
video_files=[video_directory + '/' + f for f in os.listdir(video_directory)]
shuffle=True
n_iter=1

In [5]:
print(video_files)

['/content/gdrive/My Drive/CycleAI/20200308/2020-03-08_CycleAI_0001_Niederndodeleben_001.MP4', '/content/gdrive/My Drive/CycleAI/20200308/2020-03-08_CycleAI_0001_Niederndodeleben_002.MP4', '/content/gdrive/My Drive/CycleAI/20200308/2020-03-08_CycleAI_0001_Niederndodeleben_003.MP4', '/content/gdrive/My Drive/CycleAI/20200308/2020-03-08_CycleAI_0001_Niederndodeleben_004.MP4', '/content/gdrive/My Drive/CycleAI/20200308/2020-03-08_CycleAI_0001_Niederndodeleben.gpx']


In [0]:
class VideoPipe(Pipeline):
    def __init__(self, batch_size, num_threads, device_id, data, shuffle):
        super(VideoPipe, self).__init__(batch_size, num_threads, device_id, seed=16)
        self.input = ops.VideoReader(device="gpu", filenames=data, sequence_length=sequence_length,
                                     shard_id=0, num_shards=1,
                                     random_shuffle=shuffle, initial_fill=initial_prefetch_size)


    def define_graph(self):
        output = self.input(name="Reader")
        return output

In [0]:
pipe = VideoPipe(batch_size=batch_size, num_threads=2, device_id=0, data=video_files, shuffle=shuffle)
pipe.build()
for i in range(n_iter):
    pipe_out = pipe.run()
    sequences_out = pipe_out[0].as_cpu().as_array()
    print(sequences_out.shape)